### 1. Imports

In [34]:
import pandas as pd
import numpy as np
import os
import openai
import nltk
import random
import re
import ast
import time
import math



from nltk.tokenize import sent_tokenize
nltk.download('punkt')


# openai.api_key = # Enter your API KEY

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maxencedubois/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2. Data & Parameters

In [35]:
data = pd.read_csv('your_data.csv')
data_2 = pd.read_csv('your_data.csv')
size_answers = 1200
sent_call = 120
split_sentences = "Yes"

### 3. Defining functions

In [18]:
def remove_apostrophe(text):
    return text.replace("'", "")

def add_backslash(text):
    return text.replace("'", "\\'")

def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            flat_list.append(item)
    return flat_list

In [19]:
def getquotes(df,sent_call,split_sentences):

    if split_sentences == 'Yes':
        df['value'] = df['value'].apply(sent_tokenize)
        df = df.explode('value').reset_index(drop=True)
        df['value'] = df['value'].str.split('- ')  
        df = df.explode('value').reset_index(drop=True)
    
    df = df[df['value'].str.len()>3]
    df['value'] = df['value'].apply(remove_apostrophe)

    dfs = []

    for value in df['variable'].unique():
        new_df = df[df['variable'] == value].copy()
        dfs.append(new_df)


    output = pd.DataFrame(columns=["Question", "Best_quote1", "Best_quote2", "Best_quote3"])


    for df in dfs:

        if len(df) > sent_call:

            sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]
            n = len(sent)
            sublists = [sent[i:i+sent_call] for i in range(0, n, sent_call)]


            q = df['variable'].iloc[0]

            quotes = []

            for sublist in sublists :

                n_sub = len(sublist)

                p2 = f"This is a list of {n_sub} answers to the quesiton '{q}' '{sublist}'. Quote 3 highly representative and interesting answers that provide the most information and add them to a python list list. IMPORTANT : do not change the way the quotes are written, do not add any punctuation. Keep the sentence exactly as it is. The list is the following : "


                response2 = openai.Completion.create(
                         model="text-davinci-003",
                         prompt=p2,
                         temperature=0,
                         max_tokens=300,
                         top_p=1.0,
                         frequency_penalty=0.0,
                         presence_penalty=0.0)


                text2 = response2['choices'][0]['text'].lstrip()
                text2 = str(text2)
                text2 = ast.literal_eval(text2)

                for i in text2:
                    quotes.append(i)

            bq1 = random.choice(quotes)
            quotes.remove(bq1)

            bq2 = random.choice(quotes)
            quotes.remove(bq2)

            bq3 = random.choice(quotes)

            output.loc[len(output)] = [q,bq1, bq2, bq3]

        else:

            quotes = []

            sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]

            q = df['variable'].iloc[0]

            n = len(df)

            p2 = f"This is a list of {n} answers to the quesiton '{q}' '{sent}'. Quote 3 highly representative and interesting answers that provide the most information and add them to a python list list. IMPORTANT : do not change the way the quotes are written, do not add any punctuation. Keep the sentence exactly as it is. The list is the following : "


            response2 = openai.Completion.create(
                    model="text-davinci-003",
                         prompt=p2,
                         temperature=0,
                         max_tokens=300,
                         top_p=1.0,
                         frequency_penalty=0.0,
                         presence_penalty=0.0)


            text2 = response2['choices'][0]['text'].lstrip()
            text2 = str(text2)
            text2 = ast.literal_eval(text2)

            for i in text2:
                    quotes.append(i)          

            bq1 = quotes[0]

            bq2 = quotes[1]

            bq3 = quotes[2]

            output.loc[len(output)] = [q, bq1, bq2, bq3]
            
    return output

In [20]:
def getsummarry(df,sent_call,size_answers,split_sentences):
    
    if split_sentences == 'Yes':
        df['value'] = df['value'].apply(sent_tokenize)
        df = df.explode('value').reset_index(drop=True)
        df['value'] = df['value'].str.split('- ')  
        df = df.explode('value').reset_index(drop=True)
        df = df[df['value'].str.len()>3]
        df['value'] = df['value'].apply(remove_apostrophe)

    dfs = []

    for value in df['variable'].unique():
        new_df = df[df['variable'] == value].copy()
        dfs.append(new_df)

    
    output2 = pd.DataFrame(columns=["Question", "Summary"])
    
    res_max = math.floor(3200/(math.ceil(size_answers / sent_call)))-30


    for df in dfs:

        if len(df) > sent_call:

            sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]
            n = len(sent)
            sublists = [sent[i:i+sent_call] for i in range(0, n, sent_call)]


            q = df['variable'].iloc[0]

            sumarries = []
            

            for sublist in sublists :

                n_sub = len(sublist)

                p = f" This is a list of {n_sub} answers to the question '{q}'. '{sublist}' Give me thorough detailed analysis of the answers provided. Including examples defending what you highlight as well as the important topics covered. No introduction, no conclusion, only analysis."


                response = openai.Completion.create(
                             model="text-davinci-003",
                             prompt=p,
                             temperature=0,
                             max_tokens=res_max,
                             top_p=1.0,
                             frequency_penalty=0.0,
                             presence_penalty=0.0
                            )

                text = response['choices'][0]['text'].lstrip()

                sumarries.append(text)


            n_sum = len(sumarries)

            p = f" This is a list of {n_sum} summaries. Each summary represents an analysis of up to 150 responses to my survey on the question '{q}'.Can you put these '{n}' summaries together to give me a detailed overview of the answers?  The analysis should be detailed, show examples of what you defend, and highlight the major topics covered in the answers.'{sumarries}'"

            response = openai.Completion.create(
                    model="text-davinci-003",
                        prompt=p,
                        temperature=0,
                        max_tokens=800,
                        top_p=1.0,
                        frequency_penalty=0.0,
                        presence_penalty=0.0
                        )

            text = response['choices'][0]['text'].lstrip()

            output2.loc[len(output2)] = [q, text]

        else:


            sent = [f"{index + 1}. {sentence}" for index, sentence in enumerate(df['value'])]

            q = df['variable'].iloc[0]

            n = len(df)

            p = f" This is a list of {n} answers to the question '{q}'. '{sent}' Give me thorough detailed analysis of the answers provided. Including examples defending what you highlight as well as the important topics covered."    

            response = openai.Completion.create(
                    model="text-davinci-003",
                        prompt=p,
                        temperature=0,
                        max_tokens=1000,
                        top_p=1.0,
                        frequency_penalty=0.0,
                        presence_penalty=0.0
                        )

            text = response['choices'][0]['text'].lstrip()

            output2.loc[len(output2)] = [q, text]
    
    
    return output2


### 4. Results

In [36]:
def getResults(data,data_2,sent_call,size_answers,split_sentences): 
    output = getquotes(data,sent_call,split_sentences)
    output2 = getsummarry(data2,sent_call,size_answers,split_sentences)
    merged_df = pd.merge(output, output2, on='Question')
    merged_df.to_excel('merged_output.xlsx', index=False)
    merged_df.to_csv('merged_output.csv', index=False)
    
    return merged_df    

In [37]:
merged_df = getResults(data,data_2,sent_call,size_answers,split_sentences)